# Hair Classifier Homework Notebook

## Install dependencies

In [1]:

!pip install onnx onnxruntime pillow numpy requests


## Download model files

In [2]:

import urllib.request

PREFIX="https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"

files = [
    "hair_classifier_v1.onnx",
    "hair_classifier_v1.onnx.data"
]

for f in files:
    print("Downloading:", f)
    urllib.request.urlretrieve(f"{PREFIX}/{f}", f)


Downloading: hair_classifier_v1.onnx
Downloading: hair_classifier_v1.onnx.data


## Question 1 — Output node name

In [3]:

import onnx

model = onnx.load("hair_classifier_v1.onnx")

for o in model.graph.output:
    print("Output node:", o.name)


Output node: output


## Question 2 — Target size

In [4]:

TARGET_SIZE = (200, 200)
print("Target size:", TARGET_SIZE)


Target size: (200, 200)


## Helper: download & preprocess

In [6]:

from urllib.request import urlopen
from io import BytesIO
from PIL import Image
import numpy as np

def download_and_preprocess(url):
    data = urlopen(url).read()
    img = Image.open(BytesIO(data))

    img = img.resize(TARGET_SIZE, Image.NEAREST)

    x = np.array(img).astype("float32")
    x = (x - 127.5) / 127.5
    x = np.expand_dims(x, 0)

    return x, img


## Question 3 — First pixel value (R)

In [7]:

url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
x, img = download_and_preprocess(url)

print("First pixel R:", x[0,0,0])


First pixel R: [-0.52156866 -0.18431373 -0.827451  ]


## Question 4 — Model output and probability

In [8]:

import onnxruntime as ort
import math

sess = ort.InferenceSession("hair_classifier_v1.onnx")

input_name = sess.get_inputs()[0].name

if sess.get_inputs()[0].shape[1] == 3:
    x_in = np.transpose(x, (0, 3, 1, 2))
else:
    x_in = x

logit = sess.run(None, {input_name: x_in})[0].squeeze()

prob = 1 / (1 + math.exp(-logit))

print("Logit:", logit)
print("Probability:", prob)


Logit: -0.20132342
Probability: 0.4498384560019909



## Question 5 and 6 — Docker and container runs

These will be run in a terminal, not inside the notebook:

```bash
docker images
# Find: agrigorev/model-2025-hairstyle:v1 -> SIZE: 608MB
```

```bash
docker run -it --rm -p 8080:8080 hairstyle-lambda

curl http://localhost:8080/2015-03-31/functions/function/invocations   -d '{"url": "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"}'
```
